In [1]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Krish Naik is a YouTuber and educator known for his work in the field of data science and machine learning. He creates educational content on platforms like YouTube, where he shares tutorials, courses, and insights related to data science, artificial intelligence, and programming. His videos often cover topics such as Python programming, machine learning algorithms, deep learning, and data visualization. Krish Naik is recognized for his engaging teaching style and practical approach to complex subjects, making them accessible to a wide audience.",
        metadata={
            "source": "https://www.youtube.com/@KrishNaik",
            "title": "Krish Naik",
        }),
    Document(
        page_content="I like Asian Baddies",
        metadata={
            "source": "My Brain",
            "title": "My preference",
        }),
    Document(
        page_content="I am talking to a few girls right now",
        metadata={
            "source": "My Whatsapp chats",
            "title": "Whatsapp"
        }),
    Document(
        page_content="Why am I like this? I don't know. The past has moulded me into this person. I am a product of my past experiences and choices.",
        metadata={
            "source": "My life experiences",
            "title": "My Life"})
]

In [2]:
documents

[Document(metadata={'source': 'https://www.youtube.com/@KrishNaik', 'title': 'Krish Naik'}, page_content='Krish Naik is a YouTuber and educator known for his work in the field of data science and machine learning. He creates educational content on platforms like YouTube, where he shares tutorials, courses, and insights related to data science, artificial intelligence, and programming. His videos often cover topics such as Python programming, machine learning algorithms, deep learning, and data visualization. Krish Naik is recognized for his engaging teaching style and practical approach to complex subjects, making them accessible to a wide audience.'),
 Document(metadata={'source': 'My Brain', 'title': 'My preference'}, page_content='I like Asian Baddies'),
 Document(metadata={'source': 'My Whatsapp chats', 'title': 'Whatsapp'}, page_content='I am talking to a few girls right now'),
 Document(metadata={'source': 'My life experiences', 'title': 'My Life'}, page_content="Why am I like th

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

In [5]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/jyo/Desktop/Projects/KN_Agents/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(documents, embedding=embeddings)

/Users/jyo/Desktop/Projects/KN_Agents/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


### Retriever

In [10]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['Asia','life'])

/Users/jyo/Desktop/Projects/KN_Agents/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[Document(id='bf20ab26-c44a-4503-ad55-4b5eec82576f', metadata={'source': 'My Brain', 'title': 'My preference'}, page_content='I like Asian Baddies')],
 [Document(id='4c442210-41d9-4e67-b916-41114ca87512', metadata={'source': 'https://www.youtube.com/@KrishNaik', 'title': 'Krish Naik'}, page_content='Krish Naik is a YouTuber and educator known for his work in the field of data science and machine learning. He creates educational content on platforms like YouTube, where he shares tutorials, courses, and insights related to data science, artificial intelligence, and programming. His videos often cover topics such as Python programming, machine learning algorithms, deep learning, and data visualization. Krish Naik is recognized for his engaging teaching style and practical approach to complex subjects, making them accessible to a wide audience.')]]

In [11]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1}
)

retriever.batch(['Asia','life'])

/Users/jyo/Desktop/Projects/KN_Agents/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


[[Document(id='bf20ab26-c44a-4503-ad55-4b5eec82576f', metadata={'title': 'My preference', 'source': 'My Brain'}, page_content='I like Asian Baddies')],
 [Document(id='4c442210-41d9-4e67-b916-41114ca87512', metadata={'source': 'https://www.youtube.com/@KrishNaik', 'title': 'Krish Naik'}, page_content='Krish Naik is a YouTuber and educator known for his work in the field of data science and machine learning. He creates educational content on platforms like YouTube, where he shares tutorials, courses, and insights related to data science, artificial intelligence, and programming. His videos often cover topics such as Python programming, machine learning algorithms, deep learning, and data visualization. Krish Naik is recognized for his engaging teaching style and practical approach to complex subjects, making them accessible to a wide audience.')]]

In [13]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x17e4be3c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x17e4bef90>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [16]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the question based on the context provided. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{question}

Context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()}| prompt | model

response = rag_chain.invoke("what do i like?")
print(response.content)

/Users/jyo/Desktop/Projects/KN_Agents/.venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Based on the provided text, you like Asian Baddies. 

